In [9]:
import numpy as np
import pandas as pd
from PIL import Image

In [131]:
from cmath import exp


class Neuron:
    def __init__(self, activation_function=lambda x: x, learning_rate = 0.001):
        self.activation_function = activation_function
        self.learning_rate = learning_rate
        self.weights = []
    
    def set_weights(self, n: int):
        for _ in range(n):
            self.weights.append(np.random.randn())
      

    def set_weights_input_layer(self):
        self.weights = [1]
    
    def proceed(self,inputs):
        return self.activation_function(sum([self.weights[i] * inputs[i] for i in range(len(inputs))]))

    def recompute_weights(self, error, input_line):
        for i in range(len(self.weights)):
            self.weights[i] = self.weights[i] - error * input_line[i] * self.learning_rate

    def get_err_part(self, error, i: int):
        total = sum([abs(self.weights[i]) for i in range(len(self.weights))])
        return error * abs(self.weights[i]) / total
        # total = sum([self.weights[i] for i in range(len(self.weights))])
        # return error * self.weights[i] / total

class ReteaNeuronala:
    def __init__(self, hidden_layers, epochs, activation_function = lambda x : x, learning_rate = 0.001) -> None:
        self.layers = [[Neuron(activation_function, learning_rate) for _ in range(layer)] for layer in hidden_layers]
        self.epochs = epochs
    
    def train(self,input_set, output_set):
        self.layers =  [[Neuron() for _ in range(len(input_set[-1]))]] + self.layers
        self.layers = self.layers + [[Neuron(activation_function=lambda x : 1/(1+exp(-x))) for _ in range(len(set(output_set)))]]

        for i in range(len(self.layers)):
            for neuron in self.layers[i]:
                if i == 0:
                    neuron.set_weights_input_layer()
                else:
                    neuron.set_weights(len(self.layers[i-1]))

        for _ in range(self.epochs):
            print("STRAT A NEW EPOCH")
            for i in range(len(input_set)):
                input_line = input_set[i]
                output = output_set[i]
                result = self.compute(input_line)
                if output == 'YES':
                    self.balance_weights(0,1-result[0],input_line, len(self.layers)-1)
                    self.balance_weights(1,0-result[1],input_line, len(self.layers)-1)
                else:
                    self.balance_weights(0,0-result[0],input_line, len(self.layers)-1)
                    self.balance_weights(1,1-result[1],input_line, len(self.layers)-1)

            print(self.layers[-1][0].weights)
            print(self.layers[-1][1].weights)


    def balance_weights(self,neuron_index, error, input_line, layer_index):
        self.layers[layer_index][neuron_index].recompute_weights(error,input_line)
        if layer_index - 1 != 0:
            for j in range(len(self.layers[layer_index-1])):
                err = self.layers[layer_index][neuron_index].get_err_part(error, j)
                self.balance_weights(j,err,input_line,layer_index-1)

    def compute(self, input_line):
        result = input_line
        for layer in self.layers[1:]:
            result = [layer[i].proceed(result) for i in range(len(layer))]
        return result
    
    def predict(self, inputs):
        outputs = []
        for input in inputs:
            outputs.append(self.compute(input))
        return outputs

In [132]:
def read_datas() -> pd.DataFrame:
    df = pd.read_csv('datas.csv')
    df = df.dropna()

    return df

def getTrainingAndValidationDatas():
    np.random.seed(5)
    df = read_datas()
    n = df.shape[0]
    indexes = [i for i in range(n)]
    trainingIndexes = np.random.choice(indexes, int(0.7 * n), replace = False)
    validationIndexes = [i for i in range(n) if not i in trainingIndexes]

    trainingInputs = [df['Photo'].iloc[i] for i in trainingIndexes]
    trainingOutputs = [df['Has Filter'].iloc[i] for i in trainingIndexes]

    validationInputs = [df['Photo'].iloc[i] for i in validationIndexes]
    validationOutputs = [df['Has Filter'].iloc[i] for i in validationIndexes]

    return trainingInputs, trainingOutputs, validationInputs, validationOutputs

def getInputParameters(inputImages, size):
    params = []
    for imagePath in inputImages:
        params.append([1])
        image = Image.open(imagePath)
        image = image.resize(size)
        for pixel in list(image.getdata()):
            r,g,b = pixel[0],pixel[1],pixel[2]
            params[-1].append(r)
            params[-1].append(g)
            params[-1].append(b)
    return params

def tang_func(x):
    from numpy import tanh
    return tanh(x)

def defineClassifier(hidden_layers, activation_function, trainInputs, trainOutputs):
    classifier = ReteaNeuronala(hidden_layers,5,activation_function)
    classifier.train(trainInputs, trainOutputs)
    return classifier

def testClassifier(hidden_layers,activation_function,size):
    trainingInputSet, trainingOutputSet, validationInputSet, validationOutputSet = getTrainingAndValidationDatas()
    trainInputs = getInputParameters(trainingInputSet, size)
    trainOutputs = trainingOutputSet
    classifier = defineClassifier(hidden_layers, activation_function, trainInputs, trainOutputs)

    validationInputs = getInputParameters(validationInputSet, size)
    outputs = classifier.predict(validationInputs)

    for v1,v2 in zip(outputs, validationOutputSet):
        print(v1,v2)

testClassifier([10,2,5,3],tang_func,(64,64))

STRAT A NEW EPOCH
[(0.28340858486094433+0j), (1.5121556854884028+0j), (0.3626663589330312+0j)]
[(0.5697207050227878+0j), (0.0022135218418769947+0j), (0.143285568281497+0j)]
STRAT A NEW EPOCH
[(0.28804968740450526+0j), (1.504388147637028+0j), (0.7208020773928244+0j)]
[(0.5662914320760218+0j), (0.35993615659285116+0j), (0.05789985596529537+0j)]
STRAT A NEW EPOCH
[(0.28770046269277955+0j), (0.6544303685210153+0j), (0.3468459060356402+0j)]
[(0.5634862676571483+0j), (0.7621808045034257+0j), (0.02904707351127192+0j)]
STRAT A NEW EPOCH
[(0.2936851098589195+0j), (0.6932130996967497+0j), (0.7777415900762016+0j)]
[(0.5545766133222726+0j), (0.2827691118963439+0j), (-0.7854135222022287+0j)]
STRAT A NEW EPOCH
[(0.297211589431467+0j), (0.3930965717377327+0j), (0.8982004895205948+0j)]
[(0.5299788859715298+0j), (-2.4077145795689585+0j), (-3.5967584516850755+0j)]
[(0.5517867025613467+0j), (0.1519970822976643+0j)] NO
[(0.5517867025613467+0j), (0.1519970822976643+0j)] NO
[(0.5517867025613467+0j), (0.1519